## Crypto Market

### Coingecko

In [4]:
import requests
import pandas as pd

def get_coingecko_data(days='max'):
    url = "https://api.coingecko.com/api/v3/coins/bitcoin/market_chart"
    params = {
        'vs_currency': 'usd',
        'days': days,
        'interval': 'daily'
    }
    response = requests.get(url, params=params)
    data = response.json()
    
    prices = pd.DataFrame(data['prices'], columns=['timestamp', 'price'])
    market_caps = pd.DataFrame(data['market_caps'], columns=['timestamp', 'market_cap'])
    volumes = pd.DataFrame(data['total_volumes'], columns=['timestamp', 'volume'])
    
    df = prices.merge(market_caps, on='timestamp').merge(volumes, on='timestamp')
    df['date'] = pd.to_datetime(df['timestamp'], unit='ms')
    return df.set_index('date')

# Usage
btc_data = get_coingecko_data(days=365)

In [5]:
btc_data

,timestamp,price,market_cap,volume
date,,,,
2024-02-10 00:00:00,1707523200000,47143.242991,9.255419e+11,4.199395e+10
2024-02-11 00:00:00,1707609600000,47768.995251,9.382108e+11,1.646116e+10
2024-02-12 00:00:00,1707696000000,48189.881793,9.455796e+11,1.319688e+10
2024-02-13 00:00:00,1707782400000,50050.136095,9.814634e+11,3.768444e+10
2024-02-14 00:00:00,1707868800000,49732.145370,9.745853e+11,3.733108e+10
...,...,...,...,...
2025-02-05 00:00:00,1738713600000,98118.439217,1.943535e+12,7.319669e+10
2025-02-06 00:00:00,1738800000000,96582.886829,1.912585e+12,4.884896e+10
2025-02-07 00:00:00,1738886400000,96635.418753,1.915094e+12,4.446218e+10


### Binance

In [6]:
import pandas as pd
from binance.spot import Spot as Client
from binance.lib.utils import config_logging
import logging
from datetime import datetime

In [22]:
symbol_map = {
    "bitcoin": "BTCUSDT",
    "ethereum": "ETHUSDT",
    "binance-coin": "BNBUSDT",
    "ripple": "XRPUSDT",
    "cardano": "ADAUSDT",
    "solana": "SOLUSDT",
    "polkadot": "DOTUSDT",
    "dogecoin": "DOGEUSDT",
    "shiba-inu": "SHIBUSDT",
    "litecoin": "LTCUSDT",
    "chainlink": "LINKUSDT",
    "polygon": "MATICUSDT",
    "avalanche": "AVAXUSDT",
    "uniswap": "UNIUSDT",
    "cosmos": "ATOMUSDT",
    "stellar": "XLMUSDT",
    "vechain": "VETUSDT",
    "filecoin": "FILUSDT",
    "algorand": "ALGOUSDT",
    "monero": "XMRUSDT",
    "bitcoin-cash": "BCHUSDT",
    "eos": "EOSUSDT",
    "tezos": "XTZUSDT",
    "aave": "AAVEUSDT",
    "compound": "COMPUSDT",
    "maker": "MKRUSDT",
    "theta": "THETAUSDT",
    "axie-infinity": "AXSUSDT",
    "decentraland": "MANAUSDT",
    "the-sandbox": "SANDUSDT",
    "gala": "GALAUSDT",
    "enjin-coin": "ENJUSDT",
    "curve-dao-token": "CRVUSDT",
    "yearn-finance": "YFIUSDT",
    "synthetix": "SNXUSDT",
    "uma": "UMAUSDT",
    "balancer": "BALUSDT",
    "ren": "RENUSDT",
    "0x": "ZRXUSDT",
    "loopring": "LRCUSDT",
    "harmony": "ONEUSDT",
    "kava": "KAVAUSDT",
    "zilliqa": "ZILUSDT",
    "icon": "ICXUSDT",
    "ontology": "ONTUSDT",
    "qtum": "QTUMUSDT",
    "iost": "IOSTUSDT",
    "waves": "WAVESUSDT",
    "omisego": "OMGUSDT",
    "nano": "NANOUSDT",
    "ravencoin": "RVNUSDT",
    "helium": "HNTUSDT",
    "chiliz": "CHZUSDT",
    "ankr": "ANKRUSDT",
    "fetch-ai": "FETUSDT",
    "numeraire": "NMRUSDT",
    "reserve-rights": "RSRUSDT",
    "band-protocol": "BANDUSDT",
    "ocean-protocol": "OCEANUSDT",
    "arweave": "ARUSDT",
    "celo": "CELOUSDT",
    "uma": "UMAUSDT",
    "mirror-protocol": "MIRUSDT",
    "injective-protocol": "INJUSDT",
    "alpha-finance": "ALPHAUSDT",
    "serum": "SRMUSDT",
    "raydium": "RAYUSDT",
    "orca": "ORCAUSDT",
    "audius": "AUDIOUSDT",
    "tellor": "TRBUSDT",
    "uma": "UMAUSDT",
    "perpetual-protocol": "PERPUSDT",
    "dydx": "DYDXUSDT",
    "illuvium": "ILVUSDT",
    "radix": "XRDUSDT",
    "mina": "MINAUSDT",
    "secret": "SCRTUSDT",
    "juno": "JUNOUSDT",
    "osmosis": "OSMOUSDT",
    "evmos": "EVMOSUSDT",
    "stargaze": "STARSUSDT",
    "persistence": "XPRTUSDT",
    "regen": "REGENUSDT",
    "akash": "AKTUSDT",
    "sentinel": "DVPNUSDT",
    "irisnet": "IRISUSDT",
    "kava": "KAVAUSDT",
    "terra": "LUNAUSDT",  # Note: Terra (LUNA) has been relaunched after the crash
    "anchor-protocol": "ANCUSDT",
    "mirror-protocol": "MIRUSDT",
    "thorchain": "RUNEUSDT",
    "kadena": "KDAUSDT",
    "hedera-hashgraph": "HBARUSDT",
    "elrond": "EGLDUSDT",
    "near-protocol": "NEARUSDT",
    "flow": "FLOWUSDT",
    "klaytn": "KLAYUSDT",
    "fantom": "FTMUSDT",
    "pancakeswap": "CAKEUSDT",
    "sushi": "SUSHIUSDT",
    "1inch": "1INCHUSDT",
    "bancor": "BNTUSDT",
    "kyber-network": "KNCUSDT",
    "uma": "UMAUSDT",
    "uma": "UMAUSDT",
    "uma": "UMAUSDT",
}

In [29]:
client = Client()

def get_order_book(symbol='BTCUSDT', limit=5000):
    """Get order book depth data"""
    return client.depth(symbol, limit=limit)

def get_recent_trades(symbol='BTCUSDT', limit=1000):
    """Get recent trades data"""
    return client.trades(symbol, limit=limit)

def get_historical_trades(symbol='BTCUSDT', limit=1000, from_id=None):
    """Get older historical trades"""
    return client.historical_trades(symbol, limit=limit, fromId=from_id)

def get_agg_trades(symbol='BTCUSDT', start_time=None, end_time=None, limit=1000):
    """Get compressed aggregate trades"""
    params = {'symbol': symbol, 'limit': limit}
    if start_time:
        params['startTime'] = int(pd.Timestamp(start_time).timestamp() * 1000)
    if end_time:
        params['endTime'] = int(pd.Timestamp(end_time).timestamp() * 1000)
    return client.agg_trades(**params)

def get_klines(symbol='BTCUSDT', interval='1d', start_time=None, end_time=None, limit=1000):
    """Get OHLCV candlestick data"""
    params = {
        'symbol': symbol,
        'interval': interval,
        'limit': limit
    }
    if start_time:
        params['startTime'] = int(pd.Timestamp(start_time).timestamp() * 1000)
    if end_time:
        params['endTime'] = int(pd.Timestamp(end_time).timestamp() * 1000)
    
    klines = client.klines(**params)
    columns = [
        'open_time', 'open', 'high', 'low', 'close', 'volume',
        'close_time', 'quote_volume', 'trades',
        'taker_buy_base', 'taker_buy_quote', 'ignore'
    ]
    df = pd.DataFrame(klines, columns=columns)
    df['open_time'] = pd.to_datetime(df['open_time'], unit='ms')
    df['close_time'] = pd.to_datetime(df['close_time'], unit='ms')
    numeric_cols = ['open', 'high', 'low', 'close', 'volume', 
                   'quote_volume', 'trades', 'taker_buy_base', 'taker_buy_quote']
    df[numeric_cols] = df[numeric_cols].apply(pd.to_numeric)
    return df

def get_24hr_ticker(symbol='BTCUSDT'):
    """Get 24-hour price change statistics"""
    return client.ticker_24hr(symbol)

def get_all_prices():
    """Get latest prices for all symbols"""
    return client.ticker_price()

def get_order_book_tickers():
    """Get best price/qty for all symbols"""
    return client.ticker_book_ticker()

def get_rolling_window_stats(symbol='BTCUSDT', window_size='1d'):
    """Get rolling window price statistics"""
    return client.ticker(symbol, windowSize=window_size)


In [30]:
ohlcv_data = get_klines(interval='1m', limit=1000)
print(f"OHLCV Data (head) :\n{ohlcv_data.head()}")
print(f"\n\nOHLCV Data (Tail) :\n{ohlcv_data.tail()}")

OHLCV Data (head) :
            open_time      open      high       low     close   volume  \
0 2025-02-08 03:36:00  96677.65  96683.62  96658.00  96683.61  2.66003   
1 2025-02-08 03:37:00  96683.61  96683.62  96683.61  96683.61  0.22016   
2 2025-02-08 03:38:00  96683.62  96700.00  96683.61  96699.99  1.03179   
3 2025-02-08 03:39:00  96700.00  96700.00  96677.84  96685.73  2.98074   
4 2025-02-08 03:40:00  96685.72  96693.43  96651.35  96651.35  2.62894   

               close_time   quote_volume  trades  taker_buy_base  \
0 2025-02-08 03:36:59.999  257145.912027    1237         2.05563   
1 2025-02-08 03:37:59.999   21285.864377     135         0.07991   
2 2025-02-08 03:38:59.999   99770.964002     674         0.49964   
3 2025-02-08 03:39:59.999  288211.317293     793         1.00016   
4 2025-02-08 03:40:59.999  254137.140804    1059         0.33721   

   taker_buy_quote ignore  
0    198719.888588      0  
1      7725.988074      0  
2     48313.107608      0  
3     96702.45

In [10]:
# Get order book depth
order_book = get_order_book()
bids = pd.DataFrame(order_book['bids'], columns=['price', 'quantity'], dtype=float)
asks = pd.DataFrame(order_book['asks'], columns=['price', 'quantity'], dtype=float)
logging.info(f"Order Book Top 5 Bids:\n{bids.head()}")
logging.info(f"Order Book Top 5 Asks:\n{asks.head()}")

2025-02-08 18:41:00.507 UTC INFO root: Order Book Top 5 Bids:
      price  quantity
0  96469.96   8.87224
1  96469.95   0.00012
2  96469.69   0.00024
3  96469.68   0.01319
4  96469.30   0.00012
2025-02-08 18:41:00.509 UTC INFO root: Order Book Top 5 Asks:
      price  quantity
0  96469.97   0.52167
1  96470.03   0.00030
2  96470.04   0.03514
3  96470.07   0.00018
4  96471.02   0.00018


## Market Sentiment

### Reddit

In [25]:
import praw

# Initialize PRAW with your Reddit API credentials
reddit = praw.Reddit(
    client_id="9ZaIkpznm7SHgZBrHPgt7w",
    client_secret="LptXCpj4Z9rnL_h6KrRwvGvSqTrDgQ",
    user_agent="Turbulent_Cow_1165"
)

# Scrape posts from a subreddit
subreddit = reddit.subreddit("Bitcoin")
for post in subreddit.hot(limit=10):  # Get 10 hot posts
    print(f"Title: {post.title}")
    print(f"Text: {post.selftext}")
    print(f"Upvotes: {post.score}")
    print("---------------------------")

Title: Bitcoin Newcomers FAQ - Please read!
Text: # Welcome to the /r/Bitcoin Newcomers FAQ

You've probably been hearing a lot about Bitcoin recently and are wondering what's the big deal?  Most of your questions should be answered by the resources below but if you have additional questions feel free to ask them in the comments.  

It all started with the release of **[Satoshi Nakamoto's whitepaper](https://bitcoin.org/bitcoin.pdf)** however that will probably go over the head of most readers so we recommend the following articles/books/videos as a good starting point for understanding how Bitcoin works and a little about its long term potential:

* [Article: The Bullish Case for Bitcoin](https://medium.com/@vijayboyapati/the-bullish-case-for-bitcoin-6ecc8bdecc1)
* [Book: The Bitcoin Standard](https://www.amazon.com/Bitcoin-Standard-Decentralized-Alternative-Central/dp/1119473861) - or [download a free copy here](http://cryptache.ro/wp-content/uploads/2021/04/The-Bitcoin-Standard-The-

### twitter

In [27]:
import tweepy

# Initialize Tweepy with your Twitter API credentials
client = tweepy.Client(bearer_token="AAAAAAAAAAAAAAAAAAAAAAF8ywEAAAAA0hoNDdXS14SEpuArPa80kx9Dvfc%3DBgIaZ11TXR8WNObPbQmc3M0hTp9I34sc1BzN0yturEOQnzQMJJ")

# Search for tweets containing "Bitcoin"
query = "Bitcoin lang:en -is:retweet"  # Exclude retweets
tweets = client.search_recent_tweets(query=query, max_results=10)

for tweet in tweets.data:
    print(f"Tweet: {tweet.text}")
    print("------")

Tweet: @kanyewest You don’t need Coinbase!!!!! You only need @River and you only need to buy Bitcoin. 

Kanye, just wait till you start learning what Bitcoin is and what it means to every individual on the planet.
------
Tweet: Breaking News: Join the X Token Presale!

🚀 Pinned post  ➞ https://t.co/sBHhRRZYvg

@ORACL3_agent @CaudilloUnico @IAMRAMIZ001 @FakePurple5201 @DavisIkbal @montanatnom @mazhagiah @Williamkin57 @Bitcoin_src20 @atheist_mm  2NPX6
------
Tweet: @optreal_ @Satsyxbt Why would you buy bitcoin on a brokerage when you can buy it on Venmo or Cashapp? It's money, not a security.

Sure, other exchanges are often better if you're down the rabbit hole. But for "retail", brokerage apps aren't really the way to handle money, nor do they do it well.
------
Tweet: @saylor @Strategy @Saylor, I love the idea of a strategy powered by Bitcoin! It’s all about innovation and adaptability. Speaking of which, have you checked out $Imaro? It’s a promising project with a strong community. D